In [216]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn import preprocessing

from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

## Read

In [217]:
df = pd.read_excel('data/data_for_projects.xls')

## First glance

In [218]:
columns = df.columns.values
missing = df.isna().sum().values
perc_missing = df.isna().sum().values/len(df)
unique = df.nunique().values
perc_unique = df.nunique().values/len(df)
dtypes = df.dtypes.values

stat = pd.DataFrame([missing, perc_missing, unique, perc_unique, dtypes], index=['missing', 'missing %', 'unique', 'unique %','dtype'], columns=columns)
stat = stat.round(decimals=5).astype(object)

data_statistics = pd.concat([stat, df.describe().loc[['mean', '50%','std', 'min', 'max']]]).rename({'50%': 'median'}).transpose()

sort_by = 'missing'
data_statistics.sort_values(sort_by, inplace=True)

data_statistics.round(2).head(12)

,missing,missing %,unique,unique %,dtype,mean,median,std,min,max
program_id,0,0.0,1694,0.335645,int64,4674.262334,5132.0,3058.080024,6.0,12138.0
age_group,0,0.0,2,0.000396,object,NaN,NaN,NaN,NaN,NaN
service_type_id,0,0.0,2,0.000396,int64,1.397662,1.0,0.489463,1.0,2.0
cpr_region,0,0.0,2010,0.398256,float64,3.788538,3.766474,0.574863,2.116376,5.8615
number_of_clinics_in_progam,0,0.0,93,0.018427,int64,22.492966,9.0,31.215219,1.0,215.0
target,0,0.0,4983,0.987319,object,NaN,NaN,NaN,NaN,NaN
cpr_max,1220,0.241728,304,0.060234,float64,3.502608,2.872116,2.752207,0.4317,18.958365
cpr_mean,1220,0.241728,980,0.194175,float64,2.412985,2.421375,1.431383,0.4317,8.529115
cpr_median,1220,0.241728,611,0.121062,float64,2.358973,2.392355,1.382725,0.4317,8.293353
cpr_min,1220,0.241728,204,0.04042,float64,1.336358,0.541277,1.103974,0.4317,8.293353


## Errors

In [219]:
num_mask = pd.to_numeric(df['target'], errors='coerce').isnull()
df.loc[num_mask, 'target']

49       1,3,1975
619      1,6,1988
664      1,6,5625
742       3,6,222
837      12,6,222
953     1,11,1975
1320    1,12,1975
1690     1,2,1986
2308     1,9,7678
2376      1,1,225
2449     1,4,6875
3065     1,3,1975
3080     1,1,9375
3189     22,8,222
4375     1,1,1932
4564      9,2,222
4628      1,6,225
4703     16,5,222
4891      5,6,222
Name: target, dtype: object

In [220]:
df.drop(df.loc[num_mask, 'target'].index, inplace=True)

## Data Tranformation

### One-Hot encoding

In [221]:
one_hot_encoding = pd.DataFrame()

ordinal_columns = df[['age_group']]

one_hot_encoding = pd.get_dummies(ordinal_columns, drop_first=True)
one_hot_encoding.head()

,age_group_age_group_2
0,0
1,0
2,0
3,0
4,0


In [222]:
encoded_df = df.copy()

encoded_df.drop(ordinal_columns, axis=1, inplace=True)
encoded_df = encoded_df.merge(one_hot_encoding, left_index=True, right_index=True)

encoded_df.to_csv('Preprocessed/one-hot-encoded_df.csv')

## XenIsh transformation

In [213]:
# Label encoding is used to transform non-numerical labels to numerical.
encoded_df = df.copy()

le = preprocessing.LabelEncoder()
encoded_df['program_city_group'] = le.fit_transform(encoded_df.program_city_group.values)
encoded_df['age_group'] = le.fit_transform(encoded_df.age_group.values)
encoded_df['gender_group'] = le.fit_transform(encoded_df.gender_group.values)
encoded_df['clinic_franchise'] = le.fit_transform(encoded_df.clinic_franchise.values)

encoded_df.target = pd.to_numeric(encoded_df.target, errors='coerce')
encoded_df.dropna(axis=0, inplace=True, how='all')
encoded_df = encoded_df[encoded_df['target'].notna()]

encoded_df.to_csv('Preprocessed/xenish-encoded_df.csv')

### Feature Selection

In [224]:
X = encoded_df.drop('target', axis=1)
y = encoded_df.target

In [225]:
num_features = 40
top_to_show = 20

feature_selection = pd.DataFrame()
feature_selection['Feature'] = X.columns
feature_selection.set_index('Feature', inplace=True)

feature_verdict = pd.DataFrame()
feature_verdict['Feature'] = X.columns
feature_verdict.set_index('Feature', inplace=True)## Correlation and Mutual Information

In [226]:
f_selector = SelectKBest(score_func=f_regression, k=num_features)

f_selector.fit(X, y)

feature_selection['F-value'] = f_selector.scores_
feature_verdict['F-value'] = f_selector.get_support()

fig = px.bar(feature_selection.sort_values(by='F-value', ascending=False).head(top_to_show), y='F-value')

fig.update_layout(
    title="Correlation F-score",
    xaxis_title="Features",
    yaxis_title="estimated F-score",
)




ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
f_selector = SelectKBest(score_func=mutual_info_regression, k=num_features)

f_selector.fit(X, y)

feature_selection['MI score'] = f_selector.scores_
feature_verdict['Mi'] = f_selector.get_support()

fig = px.bar(feature_selection.sort_values(by='MI score', ascending=False).head(top_to_show), y='MI score')
 
fig.update_layout(
    title="Mutual Information",
    xaxis_title="Features",
    yaxis_title="estimated MI values",
)




In [ ]:
from sklearn.feature_selection import RFE

estimator = RandomForestClassifier()
selector = RFE(estimator, n_features_to_select=num_features, step=1)
selector = selector.fit(X, y)

feature_selection['RFE'] = selector.ranking_
feature_verdict['RFE'] = selector.support_

fig = px.bar(feature_selection.sort_values(by='RFE').head(top_to_show), y='RFE')
 
fig.update_layout(
    title="RFE selection",
    xaxis_title="Features",
    yaxis_title="estimated RFE score",
)

### Select most relevant features

In [ ]:
feature_verdict['Total'] = feature_verdict[['F-value', 'Mi', 'RFE', 'IV']].sum(axis=1)

relevant_features = feature_verdict[feature_verdict['Total'] >= 3].copy()
selected_features = X[relevant_features.index].copy()

relevant_features.sort_values(by='Total', ascending=False).head(10)

## Imputation

In [182]:
column_nan_count = pd.DataFrame(encoded_df.isna().sum() > 0).rename({0: 'isna'}, axis=1)
nan_columns = column_nan_count[column_nan_count['isna'] == True].index

### Specific value

In [203]:
value_imputation_df = encoded_df.copy()

SPECIFIC_VALUE = 0

value_imputation_df.fillna(SPECIFIC_VALUE, inplace=True)

value_imputation_df.to_csv(f'Preprocessed/value_imputation_df_({SPECIFIC_VALUE}).csv')

value_imputation_df

,program_id,program_city_group,gender_group,service_type_id,distance_min,cpr_max,cpr_mean,cpr_median,cpr_std,cpr_min,...,visit_neuro_avg_max,visit_neuro_avg_mean,visit_neuro_avg_min,visit_oft_avg_max,visit_oft_avg_mean,visit_oft_avg_min,visit_surg_avg_max,visit_surg_avg_mean,visit_surg_avg_min,age_group_age_group_2
0,1358,2.0,0.0,1,0.0,1.687585,1.687585,1.687585,0.000000,1.687585,...,1.8,1.800000,1.80,1.80,1.60000,1.40,1.80,8.590000,6.38,0
1,1358,2.0,0.0,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.00,0.00000,0.00,0.00,0.000000,0.00,0
2,1358,2.0,1.0,1,0.0,1.975258,1.355735,0.652630,0.818169,0.481200,...,1.8,1.800000,1.80,1.80,1.60000,1.40,1.80,8.590000,6.38,0
3,1358,2.0,1.0,2,0.0,3.741448,3.485539,3.485539,0.361910,3.229630,...,0.0,0.000000,0.00,0.00,0.00000,0.00,0.00,0.000000,0.00,0
4,811,2.0,1.0,1,0.0,1.647994,1.861750,1.227380,0.348698,0.732298,...,6.5,4.911500,4.00,12.50,6.13715,3.50,6.50,5.144165,3.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042,1918,2.0,0.0,1,0.0,2.146535,1.285100,1.358695,0.837112,0.431700,...,9.9,8.272725,7.00,1.70,8.53555,6.25,9.75,7.591665,5.50,0
5043,1918,2.0,0.0,2,0.0,3.495655,3.495655,3.495655,0.000000,3.495655,...,0.0,0.000000,0.00,0.00,0.00000,0.00,0.00,0.000000,0.00,0
5044,7794,0.0,0.0,2,0.0,2.212645,2.212645,2.212645,0.000000,2.212645,...,0.0,0.000000,0.00,0.00,0.00000,0.00,0.00,0.000000,0.00,0
5045,1115,0.0,1.0,1,0.0,6.611755,3.516150,3.339455,1.383525,0.443966,...,49.9,1.985415,5.81,25.25,9.99740,5.00,37.45,9.727710,5.25,0


### Specific metric

In [206]:
metric_imputation_df = X.copy()

METRIC = 'mean'

for column in nan_columns:
    metric_value = metric_imputation_df[column].apply(METRIC)
    metric_imputation_df[column].fillna(metric_value, inplace=True)

metric_imputation_df.merge(y)
metric_imputation_df.to_csv(f'Preprocessed/metric_imputation_df_({METRIC}).csv')

metric_imputation_df

,program_id,program_city_group,gender_group,service_type_id,distance_min,cpr_max,cpr_mean,cpr_median,cpr_std,cpr_min,...,visit_neuro_avg_max,visit_neuro_avg_mean,visit_neuro_avg_min,visit_oft_avg_max,visit_oft_avg_mean,visit_oft_avg_min,visit_surg_avg_max,visit_surg_avg_mean,visit_surg_avg_min,age_group_age_group_2
0,1358,2.000000,1.0,1,349339.437583,1.687585,1.687585,1.687585,1.138614,1.687585,...,1.800000,1.800000,1.800000,1.80000,1.600000,1.400000,1.800000,8.590000,6.380000,0
1,1358,2.000000,1.0,2,349339.437583,3.507940,2.416335,2.362061,1.138614,1.337615,...,16.754996,6.526171,4.710625,17.62266,6.657954,4.766612,14.658651,6.892346,4.703852,0
2,1358,2.000000,1.0,1,349339.437583,1.975258,1.355735,0.652630,0.818169,0.481200,...,1.800000,1.800000,1.800000,1.80000,1.600000,1.400000,1.800000,8.590000,6.380000,0
3,1358,2.000000,1.0,2,349339.437583,3.741448,3.485539,3.485539,0.361910,3.229630,...,16.754996,6.526171,4.710625,17.62266,6.657954,4.766612,14.658651,6.892346,4.703852,0
4,811,2.000000,1.0,1,349339.437583,1.647994,1.861750,1.227380,0.348698,0.732298,...,6.500000,4.911500,4.000000,12.50000,6.137150,3.500000,6.500000,5.144165,3.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042,1918,2.000000,1.0,1,349339.437583,2.146535,1.285100,1.358695,0.837112,0.431700,...,9.900000,8.272725,7.000000,1.70000,8.535550,6.250000,9.750000,7.591665,5.500000,0
5043,1918,2.000000,1.0,2,349339.437583,3.495655,3.495655,3.495655,1.138614,3.495655,...,16.754996,6.526171,4.710625,17.62266,6.657954,4.766612,14.658651,6.892346,4.703852,0
5044,7794,1.892702,1.0,2,349339.437583,2.212645,2.212645,2.212645,1.138614,2.212645,...,16.754996,6.526171,4.710625,17.62266,6.657954,4.766612,14.658651,6.892346,4.703852,0
5045,1115,1.892702,1.0,1,349339.437583,6.611755,3.516150,3.339455,1.383525,0.443966,...,49.900000,1.985415,5.810000,25.25000,9.997400,5.000000,37.450000,9.727710,5.250000,0


### Dummy imputation

In [207]:
dummy_imputation_df = X.copy()
dummy_imputation = pd.DataFrame()

for column in nan_columns:
    unknown_label = column + '_UNKNOWN'
    unknown_values = encoded_df[column].isna() * 1

    dummy_imputation[unknown_label] = unknown_values


dummy_imputation_df = dummy_imputation_df.merge(dummy_imputation, left_index=True, right_index=True)
dummy_imputation_df.fillna(0, inplace=True)

dummy_imputation_df.merge(y)
dummy_imputation_df.to_csv('Preprocessed/dummy_imputation_df.csv')

dummy_imputation_df.head()

,program_id,program_city_group,gender_group,service_type_id,distance_min,cpr_max,cpr_mean,cpr_median,cpr_std,cpr_min,...,visit_neur_avg_min_UNKNOWN,visit_neuro_avg_max_UNKNOWN,visit_neuro_avg_mean_UNKNOWN,visit_neuro_avg_min_UNKNOWN,visit_oft_avg_max_UNKNOWN,visit_oft_avg_mean_UNKNOWN,visit_oft_avg_min_UNKNOWN,visit_surg_avg_max_UNKNOWN,visit_surg_avg_mean_UNKNOWN,visit_surg_avg_min_UNKNOWN
0,1358,2.0,0.0,1,0.0,1.687585,1.687585,1.687585,0.000000,1.687585,...,0,0,0,0,0,0,0,0,0,0
1,1358,2.0,0.0,2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,1,1,1,1,1,1,1,1,1
2,1358,2.0,1.0,1,0.0,1.975258,1.355735,0.652630,0.818169,0.481200,...,0,0,0,0,0,0,0,0,0,0
3,1358,2.0,1.0,2,0.0,3.741448,3.485539,3.485539,0.361910,3.229630,...,1,1,1,1,1,1,1,1,1,1
4,811,2.0,1.0,1,0.0,1.647994,1.861750,1.227380,0.348698,0.732298,...,0,0,0,0,0,0,0,0,0,0


### KNN Imputation

In [208]:
knn_imputation_df = X.copy()

imputer = KNNImputer(n_neighbors=5)

imputer.fit(knn_imputation_df)

Xtrans = imputer.transform(knn_imputation_df)

knn_imputation_df = pd.DataFrame(Xtrans, columns=knn_imputation_df.columns)

knn_imputation_df.merge(y)
knn_imputation_df.to_csv('Preprocessed/knn_imputation_df.csv')

knn_imputation_df

,program_id,program_city_group,gender_group,service_type_id,distance_min,cpr_max,cpr_mean,cpr_median,cpr_std,cpr_min,...,visit_neuro_avg_max,visit_neuro_avg_mean,visit_neuro_avg_min,visit_oft_avg_max,visit_oft_avg_mean,visit_oft_avg_min,visit_surg_avg_max,visit_surg_avg_mean,visit_surg_avg_min,age_group_age_group_2
0,1358.0,2.0,1.0,1.0,471.5718,1.687585,1.687585,1.687585,0.522952,1.687585,...,1.800,1.800000,1.8000,1.800,1.600000,1.400,1.800,8.590000,6.380,0.0
1,1358.0,2.0,1.0,2.0,303986.3760,2.016245,1.814571,1.673950,0.476823,1.561895,...,3.965,3.965000,3.9650,3.965,3.845000,3.725,4.305,7.909000,6.113,0.0
2,1358.0,2.0,1.0,1.0,471.5718,1.975258,1.355735,0.652630,0.818169,0.481200,...,1.800,1.800000,1.8000,1.800,1.600000,1.400,1.800,8.590000,6.380,0.0
3,1358.0,2.0,1.0,2.0,157391.4900,3.741448,3.485539,3.485539,0.361910,3.229630,...,3.865,3.518833,3.3650,3.835,3.421600,3.025,3.865,7.607250,6.013,0.0
4,811.0,2.0,1.0,1.0,40186.4858,1.647994,1.861750,1.227380,0.348698,0.732298,...,6.500,4.911500,4.0000,12.500,6.137150,3.500,6.500,5.144165,3.500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5023,1918.0,2.0,1.0,1.0,471.5718,2.146535,1.285100,1.358695,0.837112,0.431700,...,9.900,8.272725,7.0000,1.700,8.535550,6.250,9.750,7.591665,5.500,0.0
5024,1918.0,2.0,1.0,2.0,11128.0274,3.495655,3.495655,3.495655,0.636947,3.495655,...,11.030,6.844022,4.7900,10.490,7.208559,4.790,10.000,6.675770,4.740,0.0
5025,7794.0,1.8,1.0,2.0,141942.5582,2.212645,2.212645,2.212645,0.978500,2.212645,...,16.294,6.349601,5.2526,20.191,8.650058,5.379,12.682,7.415425,5.278,0.0
5026,1115.0,1.8,1.0,1.0,775.8804,6.611755,3.516150,3.339455,1.383525,0.443966,...,49.900,1.985415,5.8100,25.250,9.997400,5.000,37.450,9.727710,5.250,0.0
